# EDA

Importing libraries

In [ ]:
import sys
import os
import sqlite3
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath("../src"))

from utils.text_processing import clean_text, get_bert_embeddings

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andresgarciarobles/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#Getting de database
connection = sqlite3.connect('../data/raw/pubmed_data.db')
df = pd.read_sql_query("SELECT * FROM pubmed_data", connection)
connection.close()

In [4]:
#Cleaning text
df['cleaned_abstract'] = df['abstract'].apply(clean_text)

print(df['cleaned_abstract'].head())

0    recurrent highgrade intracranial malignancy gr...
1    purpose smallcell lung cancer sclc account app...
2    background androgenic alopecia aga commonly kn...
3    background neoadjuvant chemotherapy may consid...
4    background cicatricial alopecias chronic progr...
Name: cleaned_abstract, dtype: object


Since we have no labeled data I will take a non-supervised approach and make semantic clustering of the data.

In [ ]:
df['BERT_embeddings'] = df['cleaned_abstract'].apply(get_bert_embeddings)

Topic 0:
['diagnosis', 'scalp', 'disease', 'clinical', 'treatment', 'loss', 'case', 'alopecia', 'patient', 'hair']
Topic 1:
['van', 'critically', 'ill', 'modify', 'icu', 'subcutaneous', 'folfox', 'prostate', 'chop', 'cat']
Topic 2:
['hairline', 'color', 'hgps', 'recession', 'planus', 'frontotemporal', 'lichen', 'frontal', 'fibrose', 'ffa']
Topic 3:
['pruritus', 'chinese', 'cognitive', 'eastern', 'melanoma', 'braf', 'tdm', 'htra', 'carasil', 'vemurafenib']
Topic 4:
['grade', 'day', 'rate', 'cancer', 'toxicity', 'median', 'response', 'month', 'survival', 'patient']
